In [1]:
import gen as g
from tqdm import tqdm
import pickle
import os

In [2]:
# base function to run the character generation logic.

def generate():
    dead_farmers = 0
        
    recommended_species = None
    
    chosen_class = None
    
    background = None

    while not (recommended_species and chosen_class and background):
        stats, attempts = g.roll_stats()
        dead_farmers += attempts
        recommended_species = g.recommend_species(stats)  
        updated_stats = g.apply_species_bonus(stats, recommended_species)
        optimal_stats = g.sort_stats(updated_stats)    
        chosen_class = g.select_class(optimal_stats)    
        background = g.pick_background(optimal_stats)
    alignment = g.determine_alignment(dead_farmers)

    return stats, recommended_species, chosen_class, background, alignment, dead_farmers
    

In [3]:
# Run the generate function a bunch of times and keep track of selections to get a distribution.

def track_species_selection(num_iterations):
    
    species_count = {}
    classes_count = {}
    background_count = {}
    alignment_count = {}
    dead_farmers_count = {}
    species_class_count = {}
    species_background_count = {}
    class_background_count = {}
    stats_dict = {}
    for _ in (range(num_iterations)):
        stats, species, chosen_class, chosen_background, alignment, dead_farmers = generate()
        
        if species in species_count:
            species_count[species] += 1
        else:
            species_count[species] = 1
            
        if chosen_class in classes_count:
            classes_count[chosen_class] += 1
        else:
            classes_count[chosen_class] = 1
            
        if chosen_background in background_count:
            background_count[chosen_background] += 1
        else:
            background_count[chosen_background] = 1
            
        if alignment in alignment_count:
            alignment_count[alignment] += 1
        else:
            alignment_count[alignment] = 1
            
            
        if dead_farmers in dead_farmers_count:
            dead_farmers_count[dead_farmers] += 1
        else:
            dead_farmers_count[dead_farmers] = 1            
            
        if species not in species_class_count:
            species_class_count[species] = {}

        if chosen_class in species_class_count[species]:
            species_class_count[species][chosen_class] += 1
        else:
            species_class_count[species][chosen_class] = 1
            
        
        
        if species not in species_background_count:
            species_background_count[species] = {}

        if chosen_background in species_background_count[species]:
            species_background_count[species][chosen_background] += 1
        else:
            species_background_count[species][chosen_background] = 1
            
            
        if chosen_class not in class_background_count:
            class_background_count[chosen_class] = {}

        if chosen_background in class_background_count[chosen_class]:
            class_background_count[chosen_class][chosen_background] += 1
        else:
            class_background_count[chosen_class][chosen_background] = 1
            
        stats_dict[_] = stats

    return (
        species_count, classes_count, background_count, 
        alignment_count, dead_farmers_count, species_class_count, 
        species_background_count, class_background_count, stats_dict
    )

In [4]:
# Run the simulation at least 30 times to obtain enough data to calculate confidence intervals

def run_simulation(num_simulations, num_iterations):
    result = []
    for _ in tqdm(range(num_simulations)):
        results = (track_species_selection(num_iterations))
        result.append(results)
    return result

In [5]:
num_iterations = 10000
num_simulations = 50

In [6]:
## Run the simulation and save the results as a pickle

file_path = "sim_results.pkl"

# Check if the file exists
if not os.path.exists(file_path):
    results = run_simulation(num_simulations, num_iterations)
    with open(file_path, 'wb') as f:
        pickle.dump(results, f)
    print('simulation done')

else:
    print('already done')
    with open(file_path, 'rb') as f:
        results = pickle.load(f)


100%|████████████████████████████████████████| 50/50 [1:34:53<00:00, 113.88s/it]


simulation done
